# **INTRO TO MACHINE LEARNING - DECISION TREE AND LOGISTIC REGRESSION**

# To find all the materials for this workshop please see the GitHub repo here: https://github.com/aisutd/Workshop1_Chess_Fall2019

## CHESS DATASET

This is a kaggle dataset which was collected from over 20,000 games on the site Lichess.org. There are several teams on Lichess with over 1,500 players. it has 20058 rows and 16 columns.

It has the following attributes :- 
*   Game ID
*   Rated - indicates the genuinity of the game played
*   Start time
*   End time
*   Number of Turns
*   Game Status - whether the game was resigned or it resulted in checkmate
*   Winner - white / black
*   Time Increment
*   White Player ID
*   White Player Rating
*   Black Player ID
*   Black Player Rating
*   All Moves in Standard Chess Notation
*   Opening Eco (Standardised Code for any given opening)
*   Opening Name
*   Opening Ply (Number of moves in the opening phase)

Lots of information is contained within a single chess game, let alone a full dataset of multiple games. It is primarily a game of patterns, and data science is all about detecting patterns in data, which is why chess has been one of the most invested in areas of AI in the past. 
Though, we can construct many goals for this dataset, in this workshop our main aim is to predict the winner of the game at Early-game, Mid-game and End-game stages.



## Importing Libraries

Numpy and Pandas help us to manipulate data and makes it easier to use them.
Matplotlib and Seaborn helps us visualise data. Io is required to import dataset into Google Colab.

In [0]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import math
import io
from sklearn.model_selection import train_test_split

**Uploading file from the computer**

In [0]:
from google.colab import files
uploaded = files.upload()

**Loading the dataset into a pandas dataframe**

In [0]:
df = pd.read_csv(io.StringIO(uploaded['games.csv'].decode('utf-8')))
df.info()

## Data Preprocessing and Visualisation

* We will mainly see which columns are required (based on the goal and it's relevance to    the target variable - "WINNER"). 
* We will also remove those columns that are highly correlated to another column. (Correlation measures the strength of the linear relationship between two quantitative variables). So we either discard the other variable as they measure the same thing or we create a new variable that combines the effects of the highly correlated group.
* We will also play around with the feature "Moves" - try creating new features from it. 



**Dropping Columns based on their relevance to goal**

We will remove the following columns :-
* id (unique value for each row)
* rated (irrelevant)
* created_at (irrelevant)
* last_move_at (irrelevant)
* increment_code (irrelevant)
* white_id (list of names - irrelevant)
* black_id (list of names - irrelevant)
* opening_eco
* opening_name












In [0]:
#drop unneccessary columns
dropped_df = df.drop(
    ['id', 'rated', 'created_at', 'last_move_at', 'increment_code', 'white_id',
     'black_id', 'opening_eco', 'opening_name','victory_status'],
     axis=1
)
dropped_df.head()

This code allows us to drop these columns specified by name. "Axis=1" indicates that we are altering the data vertically. "inplace=True" indicates that we make these changes to the same dataframe and there's no need to create another copy. 

Thus, we are now left with a dataset of only 7 columns

We can place a **threshold on the number of moves (turns) each game** should have. Here, we keep the **threshold = 15**. So for games having turns less than 15 must be filtered out. 

In [0]:
#drop unneccessary games

index_cond = dropped_df[ dropped_df['turns'] < 15 ].index
 
# Delete these row indexes from dataFrame
dropped_df.drop(index_cond , inplace=True)
dropped_df.head()

### Working on the moves column

The moves column is represented using the algebraic notation.

![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/SCD_algebraic_notation.svg/500px-SCD_algebraic_notation.svg.png)

* Each square of the chessboard is identified by a unique coordinate pair—a letter and a number. The vertical columns of squares are labeled a through h from White's left to right. The horizontal rows of squares are numbered 1 to 8 starting from White's side of the board.
* Each piece is identified by a letter -  K for king, Q for queen, R for rook, B for bishop, and N for knight. Pawns are not identified by uppercase letters, but rather by the absence of one.
* When a piece makes a capture, an "x" is inserted immediately before the destination square. For example, Bxe5 (bishop captures the piece on e5). When a pawn makes a capture, the column from which the pawn departed is used to identify the pawn. For example, exd5 (pawn on the e-column captures the piece on d5)
* A move that places the opponent's king in check usually has the symbol "+" appended. Double check is commonly indicated the same as check.


We can extract a lot of information from this feature. 
* We can first split them into early game, mid game and end game stages. The early game moves can be seperated based on the column 'opening_ply'. The rest of the moves can be split into mid and end stages in ratio 70:30 (randomly selected). Then we can split each row into black and white moves.
* Count the number of moves made by each piece in each stage of both sides. 
* Count the number of checks made by each piece at the end of each stages.
* Also count the number of pieces captured by each piece of both sides, stage-wise.













In [0]:
# function to split game into three stages - opening, mid and end for black and 
# white
def splitMovesIntoStages(moves,opening_ply):
    count_row = 0
    early_black_row = []
    early_white_row = []
    mid_black_row = []
    mid_white_row = []
    end_black_row = []
    end_white_row = []
    
    # considering each row of the moves column
    element = [move for move in moves.split(' ')]

    # accessing the 'opening_ply' column row by row
    opening_moves = int(opening_ply)  

    rest_length = len(element) - int(opening_moves)
    mid_length = math.ceil(rest_length * 0.7)
    end_length = rest_length - mid_length
    
    # considering each element of each row of moves column and split them into 
    # stages (early, mide and black) and withn each stage into black and white
    for k in range(0, len(element)):
        # EARLY STAGE
        if k < opening_moves:

            if k % 2 == 0:  # WHITE OCCUPIES EVEN MOVES
                early_white_row.append(element[k])
            else:  # BLACK OCCUPIES ODD MOVES
                early_black_row.append(element[k])

        # MID STAGE
        elif k >= opening_moves and k < opening_moves + mid_length:
            if k % 2 == 0:
                mid_white_row.append(element[k])
            else:
                mid_black_row.append(element[k])

        # END STAGE
        else:
          if k % 2 == 0:
            end_white_row.append(element[k])
          else:
            end_black_row.append(element[k])
    
    
    return early_white_row, early_black_row, mid_white_row,mid_black_row,end_white_row,end_black_row
  

In [0]:
#split game into opening,mid and end
(dropped_df['Early_stage_white_moves'],
 dropped_df['Early_stage_black_moves'],
 dropped_df['Mid_stage_white_moves'],
 dropped_df['Mid_stage_black_moves'],
 dropped_df['End_stage_white_moves'],
 dropped_df['End_stage_black_moves']) = zip(*dropped_df.apply(lambda row: splitMovesIntoStages(row['moves'], row['opening_ply']), axis=1))

dropped_df.head()

In [0]:
#calculate no of moves for each piece
def movesByEachPiece(moves):
  
  noOfKingMoves = 0
  noOfQueenMoves = 0
  noOfRookMoves = 0
  noOfBishopMoves = 0
  noOfKnightMoves = 0
  noOfPawnMoves = 0
  
  for move in moves:
    if move.startswith('K'):
      noOfKingMoves += 1
    elif move.startswith('Q'):
      noOfQueenMoves += 1
    elif move.startswith('R'):
      noOfRookMoves += 1
    elif move.startswith('B'):
      noOfBishopMoves += 1
    elif move.startswith('N'):
      noOfKnightMoves += 1
    else:
      noOfPawnMoves += 1
  return (
      noOfKingMoves,
      noOfQueenMoves,
      noOfRookMoves,
      noOfBishopMoves,
      noOfKnightMoves,
      noOfPawnMoves
  )

In [0]:
#calculate no of moves for all pieces for white early game
(dropped_df['king_early_moves_white'],
dropped_df['queen_early_moves_white'],
dropped_df['rook_early_moves_white'],
dropped_df['bishop_early_moves_white'],
dropped_df['knight_early_moves_white'],
dropped_df['pawn_early_moves_white']) = zip(*dropped_df['Early_stage_white_moves'].apply(movesByEachPiece))

#calculate no of moves for all pieces for white mid game
(dropped_df['king_mid_moves_white'],
dropped_df['queen_mid_moves_white'],
dropped_df['rook_mid_moves_white'],
dropped_df['bishop_mid_moves_white'],
dropped_df['knight_mid_moves_white'],
dropped_df['pawn_mid_moves_white']) = zip(*dropped_df['Mid_stage_white_moves'].apply(movesByEachPiece))

#calculate no of moves for all pieces for white end game
(dropped_df['king_end_moves_white'],
dropped_df['queen_end_moves_white'],
dropped_df['rook_end_moves_white'], 
dropped_df['bishop_end_moves_white'],
dropped_df['knight_end_moves_white'],
dropped_df['pawn_end_moves_white']) = zip(*dropped_df['End_stage_white_moves'].apply(movesByEachPiece))

#calculate no of moves for all pieces for black early game
(dropped_df['king_early_moves_black'],
dropped_df['queen_early_moves_black'],
dropped_df['rook_early_moves_black'],
dropped_df['bishop_early_moves_black'],
dropped_df['knight_early_moves_black'],
dropped_df['pawn_early_moves_black']) = zip(*dropped_df['Early_stage_black_moves'].apply(movesByEachPiece))

#calculate no of moves for all pieces for black mid game
(dropped_df['king_mid_moves_black'],
dropped_df['queen_mid_moves_black'],
dropped_df['rook_mid_moves_black'],
dropped_df['bishop_mid_moves_black'],
dropped_df['knight_mid_moves_black'],
dropped_df['pawn_mid_moves_black']) = zip(*dropped_df['Mid_stage_black_moves'].apply(movesByEachPiece))

#calculate no of moves for all pieces for black mid game
(dropped_df['king_end_moves_black'],
dropped_df['queen_end_moves_black'],
dropped_df['rook_end_moves_black'],
dropped_df['bishop_end_moves_black'],
dropped_df['knight_end_moves_black'],
dropped_df['pawn_end_moves_black']) = zip(*dropped_df['End_stage_black_moves'].apply(movesByEachPiece))

In [0]:
dropped_df.head()

In [0]:
#calculate number of checks in each phase by black and white
def checksByEachSide(moves):
  checks = 0
  for move in moves:
    if '+' in move:
      checks += 1;
  return checks

In [0]:
#checks by white in each phase
dropped_df['checks_early_white'] = dropped_df['Early_stage_white_moves'].apply(checksByEachSide)
dropped_df['checks_mid_white'] = dropped_df['Mid_stage_white_moves'].apply(checksByEachSide)
dropped_df['checks_end_white'] = dropped_df['End_stage_white_moves'].apply(checksByEachSide)

#checks by black in each phase
dropped_df['checks_early_black'] = dropped_df['Early_stage_black_moves'].apply(checksByEachSide)
dropped_df['checks_mid_black'] = dropped_df['Mid_stage_black_moves'].apply(checksByEachSide)
dropped_df['checks_end_black'] = dropped_df['End_stage_black_moves'].apply(checksByEachSide)


In [0]:
dropped_df.head()

In [0]:
#calculate number of pieces captured by each piece
def capturesByEachPiece(moves):
  noOfCapturesByKing = 0
  noOfCapturesByQueen = 0
  noOfCapturesByRook = 0
  noOfCapturesByBishop = 0
  noOfCapturesByKnight = 0
  noOfCapturesByPawn = 0
  for move in moves:
    if 'x' in move:
      if move.startswith('K'):
        noOfCapturesByKing += 1
      elif move.startswith('Q'):
        noOfCapturesByQueen += 1
      elif move.startswith('R'):
        noOfCapturesByRook += 1
      elif move.startswith('B'):
        noOfCapturesByBishop += 1
      elif move.startswith('N'):
        noOfCapturesByKnight += 1
      else:
        noOfCapturesByPawn += 1
  return noOfCapturesByKing,noOfCapturesByQueen,noOfCapturesByRook,noOfCapturesByBishop,noOfCapturesByKnight,noOfCapturesByPawn

In [0]:
#calculate no of captures for all pieces for white early game
(dropped_df['king_early_captures_white'],
dropped_df['queen_early_captures_white'],
dropped_df['rook_early_captures_white'],
dropped_df['bishop_early_captures_white'],
dropped_df['knight_early_captures_white'],
dropped_df['pawn_early_captures_white']) = zip(*dropped_df['Early_stage_white_moves'].apply(capturesByEachPiece))

#calculate no of captures for all pieces for white mid game
(dropped_df['king_mid_captures_white'],
dropped_df['queen_mid_captures_white'],
dropped_df['rook_mid_captures_white'],
dropped_df['bishop_mid_captures_white'],
dropped_df['knight_mid_captures_white'],
dropped_df['pawn_mid_captures_white']) = zip(*dropped_df['Mid_stage_white_moves'].apply(capturesByEachPiece))

#calculate no of captures for all pieces for white end game
(dropped_df['king_end_captures_white'],
dropped_df['queen_end_captures_white'],
dropped_df['rook_end_captures_white'],
dropped_df['bishop_end_captures_white'],
dropped_df['knight_end_captures_white'],
dropped_df['pawn_end_captures_white']) = zip(*dropped_df['End_stage_white_moves'].apply(capturesByEachPiece))

#calculate no of captures for all pieces for black early game
(dropped_df['king_early_captures_black'],
dropped_df['queen_early_captures_black'],
dropped_df['rook_early_captures_black'],
dropped_df['bishop_early_captures_black'],
dropped_df['knight_early_captures_black'],
dropped_df['pawn_early_captures_black']) = zip(*dropped_df['Early_stage_black_moves'].apply(capturesByEachPiece))

#calculate no of captures for all pieces for black mid game
(dropped_df['king_mid_captures_black'],
dropped_df['queen_mid_captures_black'],
dropped_df['rook_mid_captures_black'],
dropped_df['bishop_mid_captures_black'],
dropped_df['knight_mid_captures_black'],
dropped_df['pawn_mid_captures_black']) = zip(*dropped_df['Mid_stage_black_moves'].apply(capturesByEachPiece))

#calculate no of captures for all pieces for black mid game
(dropped_df['king_end_captures_black'],
dropped_df['queen_end_captures_black'],
dropped_df['rook_end_captures_black'],
dropped_df['bishop_end_captures_black'],
dropped_df['knight_end_captures_black'],
dropped_df['pawn_end_captures_black']) = zip(*dropped_df['End_stage_black_moves'].apply(capturesByEachPiece))

In [0]:
dropped_df.head()

In [0]:
#drop opening_ply and all moves column
feature_extracted_df = dropped_df.drop(
    ['opening_ply','moves','Early_stage_white_moves', 'Early_stage_black_moves',
     'Mid_stage_white_moves', 'Mid_stage_black_moves','End_stage_white_moves',
     'End_stage_black_moves',],
     axis=1
)

In [0]:
print("Number of features: ",len(feature_extracted_df.columns))

In [0]:
#drop columns with only one unique value
for col in feature_extracted_df.columns:
    if len(feature_extracted_df[col].unique()) == 1:
        print(col)
        feature_extracted_df.drop(col,inplace=True,axis=1)

#Converting Non-Numeric fields to numeric 
To find correlation, we need to convert all non-numeric fields to numeric. In our dataset, only target variable - "winner" is non-numeric. We convert it to numeric using "astype" function. This assigns a number to each unique category. In our case, a number to each of black, white and draw.

In [0]:
#convert target to numerical data
feature_extracted_df['winner'] = feature_extracted_df['winner'].astype('category')
feature_extracted_df['winner'] = feature_extracted_df['winner'].cat.codes
feature_extracted_df['winner']

In [0]:
#perform correlation between columns
corr = feature_extracted_df.corr(method = "pearson")

In [0]:
plt.figure(figsize=(12,10))
sns.heatmap(corr, annot=True, cmap=plt.cm.Reds)
plt.figure().dpi

In [0]:
#correlation of winner with other columns
corr.winner

In [0]:
#Selecting highly correlated features
cor_target = abs(corr["winner"])
relevant_features = cor_target[cor_target>0.15]
relevant_features

In [0]:
#select columns that have high correlation with output 'winner' column
columns = ["winner",
           "black_rating",
           "king_end_moves_white",
           "queen_end_moves_white",
           "king_end_moves_black",
           "queen_end_moves_black",
           "checks_end_white",
           "checks_end_black",
           "queen_end_captures_white",
           "queen_end_captures_black",
           "rook_end_captures_black"
]
feature_engineered_df = feature_extracted_df[columns]

In [0]:
#display correlation between engineered features
corr_features = feature_engineered_df.corr(method="pearson")
plt.figure(figsize=(12,10))
sns.heatmap(corr_features, annot=True, cmap=plt.cm.Reds)
plt.show()

In [0]:
feature_engineered_df.head()

#Train and Test Split
The training set contains known output and the model learns on this data in order to be generalized to other data later on. We have the test dataset (or subset) in order to test our model’s prediction on this subset.

In [0]:
#split into train and test
label = feature_engineered_df['winner']
features = feature_engineered_df.drop('winner',axis=1)
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2)
print("Data before Split: ",features.shape)
print("Training data: ",X_train.shape)
print("Testing data: ",X_test.shape)

##Decision Trees

Conceptual Background
---
---
**What is a Decision Tree?**
* A graphical representation of possible solutions to a decision (based on certain conditions).
* It starts with the root and branches off into a number of leaves.
* A systematic process with a visual representation.


---
**Parts of a Decision Tree**
* Each node represents a feature or attribute.
* Each branch represents a decision or rule.
* Each leaf represents an outcome.


---

**Algorithms and Approaches to help build a Decision Tree?**
* Classification and Regression Trees (CART algorithm)
* Iterative Dichotomiser 3 (using entropy and information gain)
* For this workshop, we will be using the Decision Tree Classifier provided by SciKit-Learn.

---

**Process**
* Determine the root node. We pick the attribute that best classifies the training data.
* Top-down process
* Which attribute to choose? The one with the higest information gain.
* Information gain is defined using a measure called "entropy".
* 𝐻(𝑋)=−∑𝑛𝑖=1𝑃(𝑥𝑖)log2𝑃(𝑥𝑖)

---

**Advantages of applying Decision Trees**
* Easy usability and comprehension
* Incredibly transparent and robust


In [0]:
from sklearn.tree import DecisionTreeClassifier # imports the Decision Tree Classifier

# We have already imported other libraries and loaded our data
# We splitted our data into feature and target variables, as well as into training and test sets.

# Now, we build the model.
dtree = DecisionTreeClassifier() # creates Decision Tree classifier object

# Trains Decision Tree classifier
dtree.fit(X_train, y_train)

In [0]:
from sklearn.metrics import classification_report, accuracy_score

# Prediction of test dataset and evaluation of the model
y_pred = dtree.predict(X_test) 

# Model Accuracy
print('Accuracy: ', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

##Logistic Regression

Conceptual Background
---
---
**Assumptions**


*   Our data can be split into 2 regions
*   This split is linear, meaning no curves or folds, etc.
*   Ex: In 2 dimensions, this split would be a line, and in 3 dimensions a plane
*   Generally, we want a (n - 1) dimensional plane to split n-dimensional space into 2 classes
*   We call this (n - 1) plane the linear discriminant, because 1) it's linear, 2) it discriminates between some class A and not A

---
**Goals**


*   Given a input point **x** predict the probability that **x** belongs to the data class **y** - denoted *P<sub>y</sub>(x)*
*   Ex: Given some characteristics about a person, whether they're above or below 130 pounds
*   **x** would be the characteristics, and the possible classes would be above & below 130 pounds

---

**Process**: 
*This may be a little confusing for some, please ask me questions*
* Let's simplify our model to 2 dimensions, so we're looking for a discriminator in 1 dimension
* Because our plane is linear, it can be represented by *bias* + *C<sub>0</sub>x*
* The issue is that this linear function also has range from (-$\infty$,$\infty$)
* We need to map this to **(0, 1)** to represent a probabilitiy
* We can do this by using the Sigmoid function, shown below
* D: (-$\infty$, $\infty$) -> R: **(0, 1)** - this allows us to turn outputs from the linear discriminant into probability  *P<sub>y</sub>(x)*

![alt text](https://cdn-images-1.medium.com/max/1600/1*Xu7B5y9gp0iL5ooBj7LtWw.png "Sigmoid Function")

Importing necessary modules & Parameters
---
---
**Pipeline**

*   Describes a pipeline *(what a surprise)* of data transforms ending with an estimator of some kind
*   In our example, StandardScaler is the only transform, and LogisticRegression is the estimator
*   Useful for any kind of preprocessing required before actual training

---
**StandardScaler** 
*   Transforms all data x -> (x - u) / s
*   "Standardize features by removing the mean and scaling to unit variance" - SkLearn Doc.
*   Works by centering the sample data to a mean of 0 and squishing the data proportional to its standard deviation
*   u (mu) is the mean of the sample data
*   s is the sample standard deviation of the sample data

---

**Logistic Regression**

1.   Let k be a value known as the regularization parameter
2.   Higher values of k more heavily "punish" increases in parameter values
3.   This is useful to prevent overfitting , as increasing values of k favor generalization rather than strong data fitting 
4.   Can be thought of as painting with wide strokes rather than with a thin paint brush
5.   C = 1/k, so C is just the inverse of k. Hence higher values of C punish parameter weights less heavily, favoring data fitting

---

**Fitting** 

*   Modern libraries allow us to abstract away all the computation.
*    **lr_pipeline.fit (** *X_train*, *y_train* **) ** starts the model's process of finding a pseudo-optimal parameter set.

In [0]:
# imports

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


# Scaling Pipeline + Logistic Regression estimator
lr_pipeline = Pipeline([('scaler', StandardScaler()), ('lr', LogisticRegression(C=0.00001))])


# Model training below
lr_pipeline.fit(X_train, y_train)

Model Accuracy Testing
---

**Predict**


*   Used to test the performance of trained models on data
*   **lr_pipeline_predict** returns a vector containing the model predictions for each entry in ***X_test***
*  **accuracy_score** is used to compare the predicted values ***y_pred*** against the ground truth ***y_test***
* **classification_report** is used to summarize the data the model prediction and reality

In [0]:
y_pred = lr_pipeline.predict(X_test) # getting the list of predictions with input X_test (data model hasn't seen)
print('Accuracy: ', accuracy_score(y_test, y_pred)) # getting accuracy of the predicted output vs. ground truth
print(classification_report(y_test, y_pred))